# Classification

In this lab we would be going through:
- Logistic Regression
- Linear Discriminant Analysis
- Quadratic Discriminant Analysis

For this lab, we would examining the `Smarket` data set that contains a number of numeric variables plus a variable called `Direction` which has the two labels `Up` and `Down`. 

Our goal is to predict `Direction` using the other features

In [ ]:
library(ISLR2)

#understanding Smarket data set
names(Smarket) #columns of the data set
dim(Smarket) #dimension of the data set
summary(Smarket) 

# attach the data set to use the columns directly
attach(Smarket)

## Logistic Regression

We are using the `glm()` function (it can be used to fit many types of generalized liner models) to fit a logistic regression model in order to predict `Direction` using `Lag1` - `Lag5` and `Volume`.

We need to pass in the argument `family = binomial` to `glm()` in order run logistic regression model rather than some other type of generalized linear model.

In [ ]:
glm.fits = glm(
Direction ~ Lag1 + Lag2 + Lag3 + Lag4 + Lag5 + Volume, data = Smarket, family = binomial)

summary(glm.fits)

The smallest p-value here is associated with `Lag1`( `0.15` although a relatively large) and the negative coefficient suggests that if the market had a positive return yesterday, then it is less likely to go up today.

The `predict()` function can be used to predict the probability that the market will go up, given values of the predictors. The `type = "response"` option tells `R` to output probabilities of the form `P(Y = 1|X)`, as opposed to other information such as the logit.

If no data set is supplied to the `predict()` function, then the probabilities are computed for the training data that was used to fit the logistic regression model.

In [ ]:
glm.probs <- predict(glm.fits, type = "response")
glm.probs[1:10]
contrasts(Direction)

In order to make a prediction as to whether the market will go up or down on a particular day, we must convert these predicted probabilities into class labels, `Up` or `Down`.

The following two commands create a vector of class predictions based on whether the predicted probability of a market increase is greater than or less than `0.5`

In [ ]:
glm.pred <- rep("Down", 1250)
glm.pred[glm.probs > .5] = "Up"

table(glm.pred, Direction)

In [ ]:
mean(glm.pred == Direction)

The `mean()` function can be used to compute the fraction of days for which the prediction was correct. In this case, logistic regression correctly predicted the movement of the market `52.2 %` of the time.

As we have seen previously, the training error rate is often overly optimistic—it tends to underestimate the test error rate. In order to better assess the accuracy of the logistic regression model in this setting, we can fit the model using part of the data, and then examine how well it predicts the held out data.

In [ ]:
train <- (Year < 2005)

# Test data
Smarket.test <- Smarket[!train, ]
dim(Smarket.test)

#Train data
Smarket.train = Smarket[train, ]
dim(Smarket.train)

Direction.2005 = Direction[!train]

To fit the model using only the subset of the observations we can pass the `subset` argument to the `glm()` function along side other arguments  

In [ ]:
#Return a logistic regression model over the training subset of data
#Response: Direction; Predictors: Lag1, Lag2
Smarket.train.fit = function(){
    # your code here
    
}

#Return the predicted probabilities object based on training fit over test data set 
Smarket.test.predict = function(){
    # your code here
    
}

In [ ]:
fit = Smarket.train.fit()
coefficients = coef(fit)

#Test intercepts of the fit
stopifnot(round(coefficients['(Intercept)'],2)== 0.03)
stopifnot(round(coefficients['Lag1'],2)== -0.06)
stopifnot(round(coefficients['Lag2'],2)== -0.04)

In [ ]:
predict = Smarket.test.predict()

glm.pred = rep('Down', 252)
glm.pred[predict > .5] = 'Up'

#Test mean of prediction
stopifnot(round(mean(glm.pred == Direction.2005), 2) == 0.56)
stopifnot(round(mean(glm.pred != Direction.2005), 2) == 0.44)

## Linear Discriminant Analysis

For this, we would be using `lda()` function which is a part of `MASS` library.

In [ ]:
library(MASS)

In [ ]:
lda.fit = lda(Direction ~ Lag1 + Lag2, data = Smarket, subset = train)
lda.fit

plot(lda.fit)

The `plot()` function produces plots of the linear discriminants, obtained by computing −0.642 × `Lag1` − 0.514 × `Lag2` for each of the training obser- vations. The `Up` and `Down` observations are displayed separately

In [ ]:
#predict Direction based on the test data
lda.pred = predict(lda.fit, Smarket.test)
names(lda.pred)

`predict()` function returns a list with three elements. 
- The first element, `class`, contains LDA’s predictions about the movement of the market.
- The second element, `posterior`, is a matrix whose `kth` column contains the posterior probability that the corresponding observation belongs to the `kth` class
- Finally, `x` contains the linear discriminants, described earlier.

In [ ]:
lda.class = lda.pred$class
table(lda.class, Direction.2005)
mean(lda.class == Direction.2005)

Applying a 50 % threshold to the posterior probabilities allows us to recre- ate the predictions contained in `lda.pred$class`.

## Quadratic Discriminant Analysis

The syntax of `qda()` is identical to that of an `lda()`
The `predict()` function also works in the same fashion as for `lda()`

In [ ]:
#Return the fit using the training subset of data 
#Response: Direction; Predictors: Lag1, Lag2
qda.fit = function(){
    # your code here
    
}
names(qda.fit()) #variables of the object 

In [ ]:
#Test - Count of predictions
stopifnot(qda.fit()$counts['Down'] == 491)
stopifnot(qda.fit()$counts['Up'] == 507)

#to understand more about qda() and output values 
#?qda()

In [ ]:
#Return the predicted class by fitting the data over Smarket.test
qda.predict.class = function(){
    # your code here
    
}
table(qda.predict.class(), Direction.2005)

In [ ]:
#Test the mean value of right predictions
stopifnot(round(mean(qda.predict.class() == Direction.2005),2) == 0.60)